In [41]:
import requests
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

#from matplotlib import pyplot as plt
#%pylab inline

import base64
import json
import time
import urllib


#hostname='haswell01.eng.platformlab.ibm.com'
hostname='bjosm02.dli.com'
commonHeaders={'accept': 'application/json'}
req = requests.Session()

In [42]:
getTuneStatusUrl = 'http://{}:55557/platform/rest/deeplearning/v1/hypersearch'.format(hostname)
wmla_user = 'Admin'
wmla_pwd = 'Admin'
sigName = 'spark233'

r = req.get(getTuneStatusUrl, headers=commonHeaders, verify=False, auth=(wmla_user, wmla_pwd))
if not r.ok:
    print('check hpo task status failed: code=%s, %s'%(r.status_code, r.content))
else:
    if len(r.json()) == 0:
        print('There is no hpo task been created')
    for item in r.json():
        print('Hpo task: %s, State: %s'%(item['hpoName'], item['state']))

There is no hpo task been created


In [52]:
import tarfile
import tempfile
import os
def make_tarfile(output_filename, source_dir):
    with tarfile.open(output_filename, "w:gz") as tar:
        tar.add(source_dir, arcname=os.path.basename(source_dir))
MODEL_DIR_SUFFIX = ".modelDir.tar"
tempFile = tempfile.mktemp(MODEL_DIR_SUFFIX)
make_tarfile(tempFile, '/Users/zhuangxy/Documents/Notebooks/pytorch_hpo')
print(tempFile)
files = {'file': open(tempFile, 'rb')}

/var/folders/ld/5x0m1sgx4vgb8zgzt3ks8rd80000gn/T/tmpd6jepvmn.modelDir.tar


In [53]:
data =  {
        'modelSpec': # Define the model training related parameters
        {
            # Spark instance group which will be used to run the HPO sub-trainings. The Spark instance group selected
            # here should match the sub-training args, for example, if the sub-training args try to run a EDT job,
            # then we should put a Spark instance group with capability to run EDT job here.
            'sigName': sigName,
            
            # These are the arguments we'll pass to the execution engine; they follow the same conventions
            # of the dlicmd.py command line launcher
            #
            # See:
            #   https://www.ibm.com/support/knowledgecenter/en/SSFHA8_1.2.1/cm/dlicmd.html
            # In this example, args after --model-dir are all the required parameter for the original model itself.
            'args': '--exec-start PyTorch --cs-datastore-meta type=fs --python-version 3.6\
                     --gpuPerWorker 1 --model-main pytorch_mnist_HPO.py --model-dir pytorch_hpo\
                     --debug-level debug'
        },
        'algoDef': # Define the parameters for search algorithms
        {
            # Name of the search algorithm, one of Random, Bayesian, Tpe, Hyperband
            'algorithm': 'ExperimentGridSearch', 
            # Max running time of the hpo task in minutes, -1 means unlimited
            'maxRunTime': -1,  
            # Max number of training job to submitted for hpo task, -1 means unlimited’,
            'maxJobNum': 2,            
            # Max number of training job to run in parallel, default 1. It depends on both the
            # avaiable resource and if the search algorithm support to run in parallel, current only Random
            # fully supports to run in parallel, Hyperband and Tpe supports to to in parellel in some phase,
            # Bayesian runs in sequence now.
            'maxParalleJobNum': 1, 
            # Name of the target metric that we are trying to optimize when searching hyper-parameters.
            # It is the same metric name that the model update part 2 trying to dump.
            'objectiveMetric' : 'loss',
            # Strategy as how to optimize the hyper-parameters, minimize means to find better hyper-parameters to
            # make the above objectiveMetric as small as possible, maximize means the opposite.
            'objective' : 'minimize',
        },
    
        # Define the hyper-paremeters to search and the corresponding search space.            
        'experiments':
        [
             {
                 'id':1,
                 'hyperParams': [
                     {
                         'name': 'learning_rate',
                         'dataType': 'DOUBLE',
                         'fixedVal': '0.1'
                     },
                     {
                         'name': 'optimizer',
                         'dataType': 'STR',
                         'fixedVal': 'Adam'
                     },
                     {
                         'name': 'opt_decay',
                         'dataType': 'DOUBLE',
                         'fixedVal': '0.4882855458519536'
                     },
                     {
                         'name': 'beta1',
                         'dataType': 'DOUBLE',
                         'fixedVal': '0.7'
                     },
                     {
                         'name': 'beta2',
                         'dataType': 'DOUBLE',
                         'fixedVal': '0.8982816529517174'
                     }
                 ]
             },
            {
                 'id':2,
                 'hyperParams': [
                     {
                         'name': 'learning_rate',
                         'dataType': 'DOUBLE',
                         'fixedVal': '0.2'
                     },
                     {
                         'name': 'optimizer',
                         'dataType': 'STR',
                         'fixedVal': 'Adam'
                     }
                 ]
             }
        ]
       }
mydata={'data':json.dumps(data)}

In [54]:
startTuneUrl='http://{}:55557/platform/rest/deeplearning/v1/hypersearch'.format(hostname)
create = req.post(startTuneUrl, headers=commonHeaders, data=mydata, files=files, verify=False, auth=(wmla_user, wmla_pwd))
if not create.ok:
   print('submit tune job failed: code=%s, %s'%(create.status_code, create.content))
else:
   print('submit tune job succeed with hponame: %s'%create.json())

submit tune job succeed with hponame: Admin-hpo-6711337429212756


In [55]:
import time
#Here we just use the hpo name returned of the previous POST call
hpoName = create.json()

getHpoUrl = 'http://{}:55557/platform/rest/deeplearning/v1/hypersearch/{}'.format(hostname, hpoName)
res = req.get(getHpoUrl, headers=commonHeaders, verify=False, auth=(wmla_user, wmla_pwd))
if not res.ok:
    print('get hpo task failed: code=%s, %s'%(res.status_code, res.content))
else:
    json_out=res.json()
    
    while json_out['state'] in ['SUBMITTED','RUNNING']:
        print('Hpo task {} state {} progress {}'.format(hpoName, json_out['state'], json_out['progress']))
        res = req.get(getHpoUrl, headers=commonHeaders, verify=False, auth=('Admin', 'Admin'))
        json_out=res.json()
        time.sleep(20)
    
    print('Hpo task %s completes with state %s'%(hpoName, json_out['state']))
    #print('Best:%s'%json.dumps(json_out.get('best'), sort_keys=True, indent=4))
    print(json.dumps(res.json(), sort_keys=True, indent=4))

Hpo task Admin-hpo-6711337429212756 state RUNNING progress 0/2
Hpo task Admin-hpo-6711337429212756 state RUNNING progress 0/2
Hpo task Admin-hpo-6711337429212756 state RUNNING progress 0/2
Hpo task Admin-hpo-6711337429212756 state RUNNING progress 0/2
Hpo task Admin-hpo-6711337429212756 state RUNNING progress 0/2
Hpo task Admin-hpo-6711337429212756 state RUNNING progress 0/2
Hpo task Admin-hpo-6711337429212756 state RUNNING progress 0/2
Hpo task Admin-hpo-6711337429212756 state RUNNING progress 0/2
Hpo task Admin-hpo-6711337429212756 state RUNNING progress 0/2
Hpo task Admin-hpo-6711337429212756 state RUNNING progress 0/2
Hpo task Admin-hpo-6711337429212756 state RUNNING progress 0/2
Hpo task Admin-hpo-6711337429212756 state RUNNING progress 0/2
Hpo task Admin-hpo-6711337429212756 state RUNNING progress 0/2
Hpo task Admin-hpo-6711337429212756 state RUNNING progress 0/2
Hpo task Admin-hpo-6711337429212756 state RUNNING progress 0/2
Hpo task Admin-hpo-6711337429212756 state RUNNING progr

In [56]:
getTuneStatusUrl = 'http://{}:55557/platform/rest/deeplearning/v1/hypersearch'.format(hostname)
res = req.get(getTuneStatusUrl, headers=commonHeaders, verify=False, auth=(wmla_user, wmla_pwd))
if not res.ok:
    print('check tune job status failed: code=%s, %s'%(res.status_code, res.content))
else:
    #print(json.dumps(r.json(), sort_keys=True, indent=4))
    if len(res.json()) == 0:
        print('There is no hpo task been created')
    for item in res.json():
        #print(item['hpoName'])
        print('Hpo task: %s, State: %s'%(item['hpoName'], item['state']))
        #print('Hpo tasks detail:%s'%json.dumps(item, sort_keys=True, indent=4))

Hpo task: Admin-hpo-6710877403486750, State: FAILED
Hpo task: Admin-hpo-6711337429212756, State: FINISHED


In [31]:
deleteHpoUrl = 'http://{}:55557/platform/rest/deeplearning/v1/hypersearch'.format(hostname)
r=req.delete(deleteHpoUrl,headers=commonHeaders, verify=False, auth=(wmla_user, wmla_pwd))
if not r.ok:
    print('delete hpo task failed: code=%s, %s'%(r.status_code, r.content))
else:
    print('delete hpo task succeeds')

delete hpo task succeeds
